In [13]:
import dao.MongoTagUser as mtu
import panda.TagUser as tupd

mongotu = mtu.TagofSongUserRecom(user_id="1a78ac10b67911ed95dd00155dadb10b")
# list = mongotu.makeRecomUsersSet()
# print(list)

In [14]:
doc = mongotu.rdb.findDocument(
            collection_name="like", query=mongotu.query, projection=mongotu.projections)
# 储存请求用户的名字
mongotu.user_name = doc[0]['name']
# 随机获取一定数量的用户来计算tag 相似度
# 统一获取其他用户的信息
querys = [{'$sample': {"size": 50-1}}]
project = {'$project': mongotu.a_projections}
querys.append(project)
docs = mongotu.rdb.aggregateDocument(
     collection_name="like", querys=querys)
print(doc)

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'tags': ['爵士', '驾车', '古典', '90后', '民族', '伤感', '安静', '钢琴', '雷鬼', '学习', '怀旧', '夜晚', '拉丁', '轻音乐', '英伦', '治愈', 'R&B/Soul', '快乐', '孤独', '放松', 'Bossa Nova', '古风', '说唱', '游戏', '金属', '小语种', '吉他', '粤语', '兴奋', '舞曲', '思念', '影视原声', '朋克', '清新', '经典', '后摇', 'New Age', '日语', '流行', '乡村', '性感', '酒吧', '蓝调', '世界音乐', '民谣', '韩语', '欧美', '摇滚', '另类/独立', '浪漫', '感动', '电子', 'ACG', '旅行', '下午茶', '华语', '器乐', '运动']}]


In [15]:
# 判断 docs中是否存在 doc 并合并
docs = mongotu.isDocinDocs(doc=doc, docs=docs)
# print(docs)     

In [16]:
# 查找数据songs 便于 统计tags 出现的次数
# 提取docs 中用户的id，之后用
list_name = mongotu.getItemFromDocs(docs=docs, key='name')
list_id = mongotu.getItemFromDocs(docs=docs, key='id')
# 通过这个方法获取songs，来计算tag出现次数
docs_song = mongotu.getSongsforTagfromMongo(list_id)
# count_tag 统计
dict_tags = mongotu.makeMatricTagCountData(docs=docs_song)
# print(dict_tags)
        

In [17]:
# 根据以上docs 来生成 xy轴，矩阵
# 内容
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongotu.makeXYMatric(docs=docs)
# print(data)
        

In [18]:
# 生成pandas 表格
tudf = tupd.TUPandas(datas=list(
map(list, data.toarray())), items=mongotu.y, users=mongotu.x)



In [19]:
tudf.df 

,RuRuUIH,9D0ZA,h0fSYT,Z,axRJKQiq5,GcTaF8d,OejC0,afqmIx79w,j,3Fd,...,b,dj775w,wxVuBFlbYW,c4D,HrlIEqGS,FnJi,i7qZ,1PufH,qLy5bnF,gW
旅行,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
经典,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
金属,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
安静,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
日语,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACG,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
雷鬼,1,1,1,1,1,1,0,0,1,1,...,1,1,0,1,1,1,0,1,0,0
古典,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
古风,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [20]:
# 开始将tudf 中的数据根据上面的dict_tags中的数据进行替换
tudf = mongotu.changeDataCountTagTUDF(list_name, dict_tags, tudf)
   

In [21]:
tudf.df     

,RuRuUIH,9D0ZA,h0fSYT,Z,axRJKQiq5,GcTaF8d,OejC0,afqmIx79w,j,3Fd,...,b,dj775w,wxVuBFlbYW,c4D,HrlIEqGS,FnJi,i7qZ,1PufH,qLy5bnF,gW
旅行,2,19,1,8,6,6,15,1,18,7,...,7,5,1,15,3,9,6,16,2,1
经典,61,74,5,65,36,41,67,21,40,41,...,44,31,8,56,51,47,31,73,10,10
金属,3,6,0,10,6,8,4,1,8,6,...,4,2,2,8,5,8,7,11,2,1
安静,16,18,0,16,10,10,9,5,6,14,...,13,8,2,15,5,14,5,16,4,1
日语,29,63,5,52,44,42,45,28,34,35,...,41,34,5,63,40,58,36,70,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACG,61,76,7,76,60,56,61,34,51,42,...,55,47,6,72,54,79,58,87,9,9
雷鬼,2,5,1,3,2,4,0,0,3,3,...,3,2,0,2,2,3,0,3,0,0
古典,36,43,1,53,25,39,30,17,31,30,...,21,27,8,35,34,34,21,40,5,10
古风,31,48,3,51,40,30,37,15,36,25,...,39,28,6,29,30,36,37,47,4,6


In [22]:
# 推荐过程
similar_user = tudf.makeSimilarityWithPearson()

In [23]:
similar_user

,RuRuUIH,9D0ZA,h0fSYT,Z,axRJKQiq5,GcTaF8d,OejC0,afqmIx79w,j,3Fd,...,b,dj775w,wxVuBFlbYW,c4D,HrlIEqGS,FnJi,i7qZ,1PufH,qLy5bnF,gW
RuRuUIH,1.00,0.98,0.87,0.98,0.96,0.97,0.98,0.96,0.96,0.98,...,0.97,0.97,0.94,0.96,0.97,0.97,0.96,0.98,0.96,0.96
9D0ZA,0.98,1.00,0.89,0.98,0.97,0.98,0.98,0.96,0.97,0.98,...,0.98,0.98,0.94,0.97,0.98,0.98,0.97,0.99,0.95,0.95
h0fSYT,0.87,0.89,1.00,0.90,0.87,0.86,0.91,0.89,0.91,0.91,...,0.90,0.88,0.83,0.91,0.88,0.89,0.90,0.91,0.88,0.89
Z,0.98,0.98,0.90,1.00,0.98,0.98,0.98,0.96,0.98,0.98,...,0.97,0.99,0.94,0.97,0.97,0.98,0.98,0.99,0.95,0.96
axRJKQiq5,0.96,0.97,0.87,0.98,1.00,0.97,0.96,0.96,0.97,0.96,...,0.96,0.98,0.92,0.95,0.97,0.98,0.97,0.98,0.94,0.93
GcTaF8d,0.97,0.98,0.86,0.98,0.97,1.00,0.97,0.95,0.96,0.98,...,0.97,0.98,0.95,0.97,0.97,0.97,0.97,0.98,0.96,0.96
OejC0,0.98,0.98,0.91,0.98,0.96,0.97,1.00,0.96,0.97,0.98,...,0.97,0.97,0.94,0.98,0.97,0.98,0.97,0.98,0.97,0.96
afqmIx79w,0.96,0.96,0.89,0.96,0.96,0.95,0.96,1.00,0.94,0.95,...,0.95,0.97,0.91,0.97,0.96,0.97,0.96,0.96,0.92,0.93
j,0.96,0.97,0.91,0.98,0.97,0.96,0.97,0.94,1.00,0.97,...,0.96,0.97,0.92,0.96,0.95,0.96,0.98,0.98,0.95,0.94
3Fd,0.98,0.98,0.91,0.98,0.96,0.98,0.98,0.95,0.97,1.00,...,0.97,0.98,0.96,0.97,0.96,0.97,0.97,0.99,0.96,0.97


In [24]:
# 获取相似度最高几位用户sign 设置为 1 表示 使用Pearson相关系数获取相关性最高的几个用户
diction = tudf.makeTopNUsers(similar=similar_user,sign=1)
print(diction)
print(len(diction['RuRuUIH']))

{'RuRuUIH': ['9D0ZA', 'X2Bwh38', '6p11UivDhF', 'zhZd1J1', 'ReOewQ', 'm3NOW7Pp', 'KvC', 'uTAbZm', '9ZxdOqP', '3Fd', 'OejC0', '1PufH', 'Z', 'N2LBNEz', 'zVZzhaw', 'dj775w', 'yNHtXCz5M', 'HrlIEqGS', 'FnJi', 's7UIjlHxS', 'b', 'ps', 'H298YN', 'd', 'GcTaF8d', 'WfTR2', 'BJ1HR0', 'mdOQjX', 'qLy5bnF', 'axRJKQiq5', 'i7qZ', 'afqmIx79w', 'c4D', 'j', 'G', 'gW', 'MGrVl', 'eVbz', 'BK1076n', 'I3F0KrG3a', '1C3ijya', 'H6fLkSvrlg']}
42


In [25]:
# 利用当前用户的信息确定他们的id值形成对应的list_id
diction[mongotu.user_name] = mongotu.makeListIdWithUserName(diction[mongotu.user_name])
print(diction)
print(len(diction[mongotu.user_name]))

{'RuRuUIH': ['1a78b3d6b67911ed95dd00155dadb10b', '1a78bb2eb67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78c984b67911ed95dd00155dadb10b', '1a78f15cb67911ed95dd00155dadb10b', '1a78ba0cb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a78c308b67911ed95dd00155dadb10b', '1a78e414b67911ed95dd00155dadb10b', '1a78dab4b67911ed95dd00155dadb10b', '1a78d9f6b67911ed95dd00155dadb10b', '1a78ba8eb67911ed95dd00155dadb10b', '1a78e95ab67911ed95dd00155dadb10b', '1a79073cb67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a791542b67911ed95dd00155dadb10b', '1a78e2b6b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a78c056b67911ed95dd00155dadb10b', '1a78af76b67911ed95dd00155dadb10b', '1a78f4a4b67911ed95dd00155dadb10b', '1a791bf0b67911ed95dd00155dadb10b', '1a78ae7cb67911ed95dd00155dadb10b', '1a790d2cb67911ed95dd00155dadb10b', '1a78b034b67911

In [26]:
# 保存过程
# mongotu.saveDataSetRecomAnswer(diction)
# 改变形式，方便存储
docs = mongotu.changeDataFormat(diction=diction, list_keys=list(diction))
print(docs)
# 保存
mongotu.wdb.writeDocument(docs=docs, collection_name="recom")

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'RecomUsers': ['1a78b3d6b67911ed95dd00155dadb10b', '1a78bb2eb67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78c984b67911ed95dd00155dadb10b', '1a78f15cb67911ed95dd00155dadb10b', '1a78ba0cb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a78c308b67911ed95dd00155dadb10b', '1a78e414b67911ed95dd00155dadb10b', '1a78dab4b67911ed95dd00155dadb10b', '1a78d9f6b67911ed95dd00155dadb10b', '1a78ba8eb67911ed95dd00155dadb10b', '1a78e95ab67911ed95dd00155dadb10b', '1a79073cb67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a791542b67911ed95dd00155dadb10b', '1a78e2b6b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a78c056b67911ed95dd00155dadb10b', '1a78af76b67911ed95dd00155dadb10b', '1a78f4a4b67911ed95dd00155dadb10b', '1a791bf0b67911ed95dd00155dadb10b', '1a78ae7cb67911ed95dd0